In [1]:
#%#%#%#%#%#%#%##%#%#%#%#%#%#%##%#%#%#%#%#%#%##%#%#%#%#%#%#%# 

#%#%#%#%#%#%#%# Librerias #%#%#%#%#%#%#%#

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore",
                        category = DeprecationWarning)

#%#%#%#%#%#%#%# Lectura de la información #%#%#%#%#%#%#%#
#%#%#%#%#%#%#%# Base de datos #%#%#%#%#%#%#%#

COSTO = pd.read_excel('Base_Uso_costos.xlsx')
MATRFC = pd.read_excel('RFCMAT.xlsx')

In [2]:
#%#%#%#%#%#%#%# Función renombramiento columnas [F00] #%#%#%#%#%#%#%#

def Rename_SAT(Data, Pre, Letra):
    # Generar nuevos nombres de columnas
    Name = ['{}{}'.format(Pre, Letra.format(i+1)) for i in range(len(Data.columns))]
    # Guardar los nombres originales y nuevos en un diccionario
    Dict = dict(zip(Data.columns, Name))
    # Cambiar el nombre de todas las columnas por los nuevos nombres
    Data = Data.rename(columns=Dict)
    return Dict, Data

#%#%#%#%#%#%#%# Aplicación de función Rename_SAT #%#%#%#%#%#%#%#

Costo_dict, Costo_name = Rename_SAT(COSTO, 'C', '{}')
Mtrfc_dict, Mtrfc_name = Rename_SAT(MATRFC, 'M', '{}')

In [3]:
#%#%#%#%#%#%#%# Porcentaje de contribución por clase #%#%#%#%#%#%#%#

Costo_M = Costo_name[['C2', 'C4', 'C11', 'C13','C15', 
                      'C17', 'C19', 'C21','C23','C25',
                      'C27','C29','C31','C33']]
Costo_Mes = Costo_M.groupby(['C2', 'C4']).sum().reset_index()

resultados = Costo_Mes.copy()

# Se itera sobre cada mes y calcular el porcentaje de contribución por clase
for mes in ['C11','C13','C15','C17','C19','C21',
            'C23','C25','C27','C29','C31','C33']:
    suma_mes_por_clase = Costo_Mes.groupby('C2')[mes].transform('sum')
    resultados[mes + '_P'] = round((Costo_Mes[mes] / suma_mes_por_clase), 3)

In [4]:
#%#%#%#%#%#%#%# Porcentaje de contribución por Dimensión, Puerto y Servidor #%#%#%#%#%#%#%#

# Se agrupan los datos por las columnas 'Dimension' y 'Puerto' para poder
# contar el número de servidores para cada combinación
Servidores = MATRFC.groupby(['Dimension','Puerto'])['Servidor'].count().reset_index()

# Se calcula la proporción de servidores para cada combinación única de 'Dimension' y 'Puerto'
Servidores['PorServidor'] = (Servidores.Servidor / Servidores.Servidor.sum())

Servidor_xs = Servidores.groupby(['Dimension']).agg({'Servidor':'sum',
                                                     'PorServidor':'sum'}).reset_index()

for mes in ['C11','C13','C15','C17','C19','C21',
            'C23','C25','C27','C29','C31','C33']:
    resultados[mes + '_CS'] = (resultados[mes] / Servidor_xs.Servidor.sum())

In [5]:
#%#%#%#%#%#%#%# Arquitectura y Costos: Valores para X_i #%#%#%#%#%#%#%#

# Se genera una lista para almacenar los resultados
resultados_lista = []

# Se hace un loop iterable sobre cada clase C2
for clase_c2 in resultados.copy()['C2'].unique():
    
    # Se filtra la tabla Valor_Costo para obtener solo las filas con la clase C2 actual
    tabla1_filtrada_c2 = resultados.copy()[resultados.copy()['C2'] == clase_c2]
    
    # Se iterar sobre cada clase C4 dentro de la clase C2 actual
    for clase_c4 in tabla1_filtrada_c2['C4'].unique():
        
        # Se filtra la tabla Valor_Costo para obtener solo las filas con la clase C4 actual
        tabla1_filtrada_c4 = tabla1_filtrada_c2[tabla1_filtrada_c2['C4'] == clase_c4]
        
        # Se itera sobre cada dimensión de la tabla Servidor_xs
        for idx, fila_tabla2 in Servidor_xs.copy().iterrows():
            
            # Se multiplica el costo promedio por servidor para cada mes por el valor del servidor correspondiente de la Servidor_xs
            for mes in ['C11','C13','C15','C17','C19','C21',
                        'C23','C25','C27','C29','C31','C33']:
                resultado = tabla1_filtrada_c4[mes + '_CS'] * fila_tabla2['Servidor']
                
                # Se calcula el costo total para este mes y esta dimensión
                costo_total = resultado.sum()
                
                # Se crea el diccionario para almacenar los resultados y las clases
                resultado_dic = {
                    'C2': clase_c2,
                    'C4': clase_c4, 
                    'Dimension': fila_tabla2['Dimension'],
                    'NoServidor': fila_tabla2['Servidor'],
                    f'C_{mes}_XS': costo_total
                }
                
                # Se agrega la información del diccionario a la lista
                resultados_lista.append(resultado_dic)

# Se convierte la lista de resultados en un DataFrame
Costos_Xs = pd.DataFrame(resultados_lista)

# Se agrupa los resultados por las columnas 'C2', 'C4', 'Dimension' y 'NoServidor',
# para sumar los valores de las columnas 'Costo_C11_XS', 'Costo_C13_XS', etc.
Costos_Xs_agrupados = Costos_Xs.groupby(['C2', 'C4', 'Dimension', 'NoServidor']).sum().reset_index()

# Nos aseguramos que solo haya una fila para cada combinación de 'C2', 'C4', 'Dimension' y 'NoServidor'
Costos_Xs_agrupados = Costos_Xs_agrupados.drop_duplicates(subset=['C2', 'C4', 'Dimension', 'NoServidor'])

In [7]:
#%#%#%#%#%#%#%#  Primera parte (CA/Et)

Costos_CARO = Costos_Xs_agrupados.copy()
for columna in Costos_CARO.columns:
    if columna.startswith('C_C'):
        nueva_columna = 'CAE_' + columna.split('_')[1]
        Costos_CARO[nueva_columna] = (Costos_CARO[columna] / RepoMRFC_name.R2.nunique())

CAE_ALL = Costos_CARO[['C2', 'C4','Dimension','NoServidor',
                       'CAE_C11','CAE_C13','CAE_C15','CAE_C17','CAE_C19','CAE_C21',
                       'CAE_C23','CAE_C25','CAE_C27','CAE_C29','CAE_C31','CAE_C33']]

In [8]:
#%#%#%#%#%#%#%#  Segunda parte (CA/Et)*(RFC-rol)(AVG-rol)

CAER_ALL = CAE_ALL.copy()

for columna in CAER_ALL.columns:
    if columna.startswith('CAE_C'):
        nueva_columna = 'CAER_' + columna.split('_')[1]
        CAER_ALL[nueva_columna] = (CAER_ALL[columna])*(RepoMRFC_name.R13.nunique())*(Mean_allrol)

# Mostrar el DataFrame resultante
CAER_ALL = CAER_ALL[['C2', 'C4','Dimension','NoServidor',
                               'CAER_C11','CAER_C13','CAER_C15','CAER_C17','CAER_C19','CAER_C21',
                               'CAER_C23','CAER_C25','CAER_C27','CAER_C29','CAER_C31','CAER_C33']]

In [9]:
#%#%#%#%#%#%#%# Roles especificos en la arquitectrura

ROL_DES = ['IDC001','IDC002','IDC010','IDC011','IDC012',
           'IDC017','IDC019','IDC020','IDC022','IDC023',
           'IDC024','IDC025','IDC026','IDC032','IDC033',
           'IDC037','IDC040','IDC041','IDC042','IDC044',
           'IDC053','IDC054','IDC055','IDC057','IDC070',
           'IDC071','IDC072','IDC131','IDC212','IDC213']

ROL_FILTRA = RepoMRFC_name[RepoMRFC_name['R13'].isin(ROL_DES)]
ROL_AV = ROL_FILTRA.groupby(['R2'])['R13'].count().reset_index().R13.mean()

In [19]:
####  Primera parte (CA/Et ) para los 10531 ue usan los roles especificos solicitados.
COSTO_EMPLE = Costos_Xs_agrupados.copy()

for columna in Costos_CARO.columns:
    if columna.startswith('C_C'):
        nueva_columna = 'CAE_' + columna.split('_')[1]
        COSTO_EMPLE[nueva_columna] = (COSTO_EMPLE[columna] / ROL_FILTRA.R2.nunique())
        
CAE_EMPLE = COSTO_EMPLE[['C2', 'C4','Dimension','NoServidor',
                       'CAE_C11','CAE_C13','CAE_C15','CAE_C17','CAE_C19','CAE_C21',
                       'CAE_C23','CAE_C25','CAE_C27','CAE_C29','CAE_C31','CAE_C33']]

In [21]:
#%#%#%#%#%#%#%#  Segunda parte (CA/Et)*(RFC-rol)(AVG-rol)

CAE_EMP = CAE_EMPLE.copy()
for columna in CAE_EMP.columns:
    if columna.startswith('CAE_C'):
        nueva_columna = 'CAER_' + columna.split('_')[1]
        CAE_EMP[nueva_columna] = (CAE_EMP[columna])*(ROL_FILTRA.R13.nunique()/(ROL_AV))
        
CAE_EMP = CAE_EMP[['C2', 'C4','Dimension','NoServidor',
                               'CAER_C11','CAER_C13','CAER_C15','CAER_C17','CAER_C19','CAER_C21',
                               'CAER_C23','CAER_C25','CAER_C27','CAER_C29','CAER_C31','CAER_C33']]

### CASO PARTICULAR

In [23]:
ROLE_PERCE = ROL_FILTRA.R13.value_counts().reset_index().rename({'count' : 'R14'}, axis = 1)
ROLE_PERCE['R15'] = (ROLE_PERCE.R14 / ROLE_PERCE.R14.sum())

GRUPO = ROL_FILTRA.groupby(['R2', 'R13'])['R13'].count()
GRUPO_DF = GRUPO.reset_index(name='R16')

EMP_ROLPERCE = GRUPO_DF.merge(ROLE_PERCE, how = 'left',
                              left_on = ['R13'], right_on = ['R13'])

EMPLE_ROL = EMP_ROLPERCE.groupby(['R2']).agg({'R16':'sum',
                                               'R15':'sum'}).reset_index()
EMPLE_ROL['Tot'] = (EMPLE_ROL.R16)*(EMPLE_ROL.R15)
EMPLE_ROL = EMPLE_ROL.sort_values(by =['R16'], ascending=False)

> costo individualizado para servidor Ana Patricia G. S. en el sistema MAT-RFC

In [24]:
CASE_APGS = ['IDC001','IDC002', 'IDC010','IDC011','IDC012','IDC057']
APSG_CAER = ROLE_PERCE[ROLE_PERCE['R13'].isin(CASE_APGS)]
APSG_CAER_P = APSG_CAER.R15.sum()*APSG_CAER.R13.count()

In [26]:
####  Primera parte (CA/Et) para los 10531 ue usan los roles especificos solicitados.

COSTO_SOLICITADO = Costos_Xs_agrupados.copy()
for columna in COSTO_SOLICITADO.columns:
    if columna.startswith('C_C'):
        nueva_columna = 'CAE_' + columna.split('_')[1]
        COSTO_SOLICITADO[nueva_columna] = (COSTO_SOLICITADO[columna] / ROL_FILTRA.R2.nunique())

CAE_SOLICITADO = COSTO_SOLICITADO[['C2', 'C4','Dimension','NoServidor',
                                   'CAE_C11','CAE_C13','CAE_C15','CAE_C17','CAE_C19','CAE_C21',
                                   'CAE_C23','CAE_C25','CAE_C27','CAE_C29','CAE_C31','CAE_C33']]

In [27]:
CAER_APSG= CAE_SOLICITADO.copy()

for columna in CAER_APSG.columns:
    if columna.startswith('CAE_C'):
        nueva_columna = 'CAER_' + columna.split('_')[1]  # Para el sufijo de la columna original
        CAER_APSG[nueva_columna] = (CAER_APSG[columna])*(1/ROL_AV)*(APSG_CAER_P)


CAER_APSG = CAER_APSG[['C2', 'C4','Dimension','NoServidor',
                       'CAER_C11','CAER_C13','CAER_C15','CAER_C17','CAER_C19','CAER_C21',
                       'CAER_C23','CAER_C25','CAER_C27','CAER_C29','CAER_C31','CAER_C33']]